# SRTC Sim Visual Interface

## The "fake" version of the VIMS Cube Visual Interface.

This notebook has a simple purpose: reading csv files made from Jcubes and printing out the results. This is the counterpart to the VIMS Cube Visual INterface, except we look at SRTC++ simulation data. One of the goals is to make it so we can view a cube in color with ease. So without further ado... please insert the file to be used.

WARNING: animations exist in this notebook! remember to shut them down before running cells from the top or you might end up with problems! (If you break it, don't worry, resetting the notebook should just do it). 

WARNING: this file was not playing right with git and had to be recreated. Careful!

In [1]:
#So making a full animation is just going to be PAIN, so instead we'll have it print out the individual 
#frames for said animation instead of trying to hold the entire thing at once. 
#Which would suck.
#Good news, this means we don't have to work with the animation software!

filepathA = r"C:\Users\deran\Desktop\DooseCSV\2012_A0.0_p"
angles = ["000", "005", "010", "015", "020", "025", "030", "035", "040", "045", "050", "055", "060", "065", "070", "075",
             "080", "085", "090", "095", "100", "105", "110", "115", "120", "125", "130", "135", "140", "145", "150", "155",
             "160", "165", "170", "175", "180", "185", "190", "195", "200", "205", "210", "215", "220", "225", "230", "235",
             "240", "245", "250", "255", "260", "265", "270", "275", "280", "285", "290", "295", "300", "305", "310", "315", 
             "320", "325", "330", "335", "340", "345", "350", "355"] 
filepathB = r"_colorCCD.Jcube.csv"

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
#import mpl_interactions.ipyplot as iplt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage
from pyvims import VIMS
from matplotlib.colors import LogNorm

angleIndex = 0
while (angleIndex < len(angles)):
    
    filepath = filepathA + angles[angleIndex] + filepathB
    
    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.csv"
    cubeGeofpIRaxes = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.axes.csv"
    #Basically, we don't use axesfp if it's not needed.
    
    #Skeleton code nabbed from https://realpython.com/python-csv/
    #We need to be able to handle both local and other cubes. 
    #Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
    #it in the way the code understands.
    
    #Step 1: use the axes to determine the size of what we're dealing with.
    
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []
    
    with open(cubeAxesfp) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisCube.append(row[i])
                    elif (line_count == 1):
                        yAxisCube.append(row[i])
                    elif (line_count == 2):
                        zAxisCube.append(row[i])
                    i = i+1
                line_count += 1
    
    with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisGeoIR.append(row[i])
                    elif (line_count == 1):
                        yAxisGeoIR.append(row[i])
                    elif (line_count == 2):
                        zAxisGeoIR.append(row[i])
                    i = i+1
                line_count += 1
    
    #wvlns: (alt: 2) contains the wavelengths, equivalent to our zAxis.
    #Shape contains cube size. Probably wxy, but may be wyx? NB is bands (w) and NL is lines (x or y?)
    #lat, lon: position! 
    #ground_inc, ground_eme, ground_azi: incidence and emission AND azimuth
        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 
    
    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisCube)):
                cubeData[i][j][k] = float(row[i])
                if (math.isnan(cubeData[i][j][k])):
                    cubeData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (cubeData[i][j][k] < 0):
                    cubeData[i][j][k] = 0 #Negative values are nonsense.
                elif (cubeData[i][j][k] > 1):
                    cubeData[i][j][k] = 1 #Make saturation obvious? Keep it from overloading. 
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisCube)):
                j = 0
                k = k + 1
            line_count += 1
    
    with open(cubeGeofpIR) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisGeoIR)):
                geoIRData[i][j][k] = float(row[i])
                if (math.isnan(geoIRData[i][j][k])):
                    geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (geoIRData[i][j][k] < -1000):
                    geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisGeoIR)):
                j = 0
                k = k + 1
            line_count += 1
    
    #helpful declarations
    #Wavelength of windows in microns (um)
    windowum = [0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576]
    #Now we need to dynamically find the indeces where this is true. 
    windowInd = [0,1,2,3,4,5,6,7]
        
    def indexFromWav(wav):
        index = 0
        j = 0
        while (j < len(zAxisCube)):
            if (float(zAxisCube[j]) < wav and float(zAxisCube[j+1]) > wav):
                if (wav - float(zAxisCube[j]) < float(zAxisCube[j+1]) - wav):
                    index = j
                    #Have to make sure we pick the closest point, not just the "lower" one.
                else:
                    index = j+1
                j = len(zAxisCube)
            j = j + 1    
        return index
        
    #image dimensions are fungible. Calculate them.  
    xDim = 0
    yDim = 0
    if (len(xAxisCube) > len(yAxisCube)):
        yDim = 9 
        xDim = 9/(len(xAxisCube)/len(yAxisCube))
    else:
        xDim = 9
        yDim = 9/(len(yAxisCube)/len(xAxisCube))
    #This may seem backward, but transpositions occur so...
    
        
    print(np.amax(cubeData))
    print(np.amin(cubeData))
    temp = np.transpose(cubeData)
    
    
    fig, axs = plt.subplots(3,3, figsize = (9,9), layout = "tight") 
    
    #Windows are, according to lambertian_wholedisk:
    #0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576
    
    
    
    c1 = axs[0,0].imshow((temp[windowInd[0]]), cmap="copper")
    c2 = axs[0,1].imshow((temp[windowInd[1]]), cmap="copper")
    c3 = axs[0,2].imshow((temp[windowInd[2]]), cmap="copper")
    c4 = axs[1,0].imshow((temp[windowInd[3]]), cmap="copper")
    c5 = axs[1,2].imshow((temp[windowInd[4]]), cmap="copper")
    c6 = axs[2,0].imshow((temp[windowInd[5]]), cmap="copper")
    c7 = axs[2,1].imshow((temp[windowInd[6]]), cmap="copper")
    c8 = axs[2,2].imshow((temp[windowInd[7]]), cmap="copper")
    #fig.colorbar(c4, ax=cax, label='High') 
    #fig.tight_layout(pad=1.0)
    axs[0,0].axis('off')
    axs[0,1].axis('off')
    axs[0,2].axis('off')
    axs[1,0].axis('off')
    axs[1,1].axis('off')
    axs[1,2].axis('off')
    axs[2,0].axis('off')
    axs[2,1].axis('off')
    axs[2,2].axis('off')
    axs[1,1].axis('off')
    
    #COLOR
    
    mod = 1.
    
    # set R, G, and B
    R = temp[7]/(mod*1.12) #I am still very unsure why we don't have the 1/16 here when we have it in both places in the full thing, but whatever.
    G = temp[4]/(mod*0.22)
    B = (temp[2]-0.03)/(mod*0.37)
    
    #B's value can introduce negatives, remove them.
    i,j = 0,0
    for x in B:
        j=0
        for y in x:
            if (y < 0):
                B[i][j] = 0
            j = j+1
        i = i+1
    
    # Now we need to combine them in the correct form: xyc, that is, x, y, then color value. 
    colorData = [R,G,B] # arranged in cyx. I thought RGB needed to be transposed but evidentialy not. Hmm.
    colorData = np.transpose(colorData) #arranged in xyc
    
    image = axs[1,1].imshow(ndimage.rotate(colorData,90), origin="lower") 
    
    axs[0,0].set(title="0.93 $\mu$m")
    axs[0,1].set(title="1.08 $\mu$m")
    axs[0,2].set(title="1.27 $\mu$m")
    axs[1,0].set(title="1.59 $\mu$m")
    axs[1,1].set(title="bestcolor (5.00, 2.01, 1.27)")
    axs[1,2].set(title="2.01 $\mu$m")
    axs[2,0].set(title="2.69 $\mu$m")
    axs[2,1].set(title="2.79 $\mu$m")
    axs[2,2].set(title="5.00 $\mu$m")
    
    plt.savefig("NewDooseA0.0Ani" + angles[angleIndex] + ".png", format="png") 

    print(angleIndex)
    angleIndex = angleIndex+1

<>:230: SyntaxWarning: invalid escape sequence '\m'
<>:231: SyntaxWarning: invalid escape sequence '\m'
<>:232: SyntaxWarning: invalid escape sequence '\m'
<>:233: SyntaxWarning: invalid escape sequence '\m'
<>:235: SyntaxWarning: invalid escape sequence '\m'
<>:236: SyntaxWarning: invalid escape sequence '\m'
<>:237: SyntaxWarning: invalid escape sequence '\m'
<>:238: SyntaxWarning: invalid escape sequence '\m'
<>:230: SyntaxWarning: invalid escape sequence '\m'
<>:231: SyntaxWarning: invalid escape sequence '\m'
<>:232: SyntaxWarning: invalid escape sequence '\m'
<>:233: SyntaxWarning: invalid escape sequence '\m'
<>:235: SyntaxWarning: invalid escape sequence '\m'
<>:236: SyntaxWarning: invalid escape sequence '\m'
<>:237: SyntaxWarning: invalid escape sequence '\m'
<>:238: SyntaxWarning: invalid escape sequence '\m'
C:\Users\deran\AppData\Local\Temp\ipykernel_16516\831583137.py:230: SyntaxWarning: invalid escape sequence '\m'
  axs[0,0].set(title="0.93 $\mu$m")
C:\Users\deran\AppDa

1.0
0.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-5.5350678037761e-17..1.659775675675676].


0
1.0
0.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-6.082842576558847e-17..1.4826702702702703].


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-7.711206544034293e-17..1.8738648648648648].


4



KeyboardInterrupt



Error in callback <function flush_figures at 0x0000022D019F5E40> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [2]:
#So making a full animation is just going to be PAIN, so instead we'll have it print out the individual frames for said animation instead of trying to hold the entire thing at once. 
#Which would suck.
#Good news, this means we don't have to work with the animation software!

filepathA = r"C:\Users\deran\Desktop\DooseCSV\2012_A0.0_p"
angles = ["000", "005", "010", "015", "020", "025", "030", "035", "040", "045", "050", "055", "060", "065", "070", "075",
             "080", "085", "090", "095", "100", "105", "110", "115", "120", "125", "130", "135", "140", "145", "150", "155",
             "160", "165", "170", "175", "180", "185", "190", "195", "200", "205", "210", "215", "220", "225", "230", "235",
             "240", "245", "250", "255", "260", "265", "270", "275", "280", "285", "290", "295", "300", "305", "310", "315", 
             "320", "325", "330", "335", "340", "345", "350", "355"] 
filepathB = r"_colorCCD.Jcube.csv"

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
#import mpl_interactions.ipyplot as iplt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage
from pyvims import VIMS
from matplotlib.colors import LogNorm

angleIndex = 0
while (angleIndex < len(angles)):
    
    filepath = filepathA + angles[angleIndex] + filepathB
    
    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.csv"
    cubeGeofpIRaxes = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.axes.csv"
    #Basically, we don't use axesfp if it's not needed.
    
    #Skeleton code nabbed from https://realpython.com/python-csv/
    #We need to be able to handle both local and other cubes. 
    #Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
    #it in the way the code understands.
    
    #Step 1: use the axes to determine the size of what we're dealing with.
    
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []
    
    with open(cubeAxesfp) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisCube.append(row[i])
                    elif (line_count == 1):
                        yAxisCube.append(row[i])
                    elif (line_count == 2):
                        zAxisCube.append(row[i])
                    i = i+1
                line_count += 1
    
    with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisGeoIR.append(row[i])
                    elif (line_count == 1):
                        yAxisGeoIR.append(row[i])
                    elif (line_count == 2):
                        zAxisGeoIR.append(row[i])
                    i = i+1
                line_count += 1
    
    #wvlns: (alt: 2) contains the wavelengths, equivalent to our zAxis.
    #Shape contains cube size. Probably wxy, but may be wyx? NB is bands (w) and NL is lines (x or y?)
    #lat, lon: position! 
    #ground_inc, ground_eme, ground_azi: incidence and emission AND azimuth
        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 
    
    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisCube)):
                cubeData[i][j][k] = float(row[i])
                if (math.isnan(cubeData[i][j][k])):
                    cubeData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (cubeData[i][j][k] < 0):
                    cubeData[i][j][k] = 0 #Negative values are nonsense.
                elif (cubeData[i][j][k] > 1):
                    cubeData[i][j][k] = 1 #Make saturation obvious? Keep it from overloading. 
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisCube)):
                j = 0
                k = k + 1
            line_count += 1
    
    with open(cubeGeofpIR) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisGeoIR)):
                geoIRData[i][j][k] = float(row[i])
                if (math.isnan(geoIRData[i][j][k])):
                    geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (geoIRData[i][j][k] < -1000):
                    geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisGeoIR)):
                j = 0
                k = k + 1
            line_count += 1
    
    #helpful declarations
    #Wavelength of windows in microns (um)
    windowum = [0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576]
    #Now we need to dynamically find the indeces where this is true. 
    windowInd = [0,1,2,3,4,5,6,7]
        
    def indexFromWav(wav):
        index = 0
        j = 0
        while (j < len(zAxisCube)):
            if (float(zAxisCube[j]) < wav and float(zAxisCube[j+1]) > wav):
                if (wav - float(zAxisCube[j]) < float(zAxisCube[j+1]) - wav):
                    index = j
                    #Have to make sure we pick the closest point, not just the "lower" one.
                else:
                    index = j+1
                j = len(zAxisCube)
            j = j + 1    
        return index
        
    #image dimensions are fungible. Calculate them.  
    xDim = 0
    yDim = 0
    if (len(xAxisCube) > len(yAxisCube)):
        yDim = 9 
        xDim = 9/(len(xAxisCube)/len(yAxisCube))
    else:
        xDim = 9
        yDim = 9/(len(yAxisCube)/len(xAxisCube))
    #This may seem backward, but transpositions occur so...
    
        
    print(np.amax(cubeData))
    print(np.amin(cubeData))
    temp = np.transpose(cubeData)
    
    
    fig, axs = plt.subplots(figsize = (9,9), layout = "tight")  
    #COLOR
    
    mod = 1.
    
    # set R, G, and B
    R = temp[7]/(mod*1.12) #I am still very unsure why we don't have the 1/16 here when we have it in both places in the full thing, but whatever.
    G = temp[4]/(mod*0.22)
    B = (temp[2]-0.03)/(mod*0.37)
    
    #B's value can introduce negatives, remove them.
    i,j = 0,0
    for x in B:
        j=0
        for y in x:
            if (y < 0):
                B[i][j] = 0
            j = j+1
        i = i+1
    
    # Now we need to combine them in the correct form: xyc, that is, x, y, then color value. 
    colorData = [R,G,B] # arranged in cyx. I thought RGB needed to be transposed but evidentialy not. Hmm.
    colorData = np.transpose(colorData) #arranged in xyc
    
    image = axs.imshow(ndimage.rotate(colorData,90), origin="lower")
    
    axs.axis('off')
    #https://stackoverflow.com/questions/11837979/removing-white-space-around-a-saved-image
    plt.gca().set_axis_off()
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.margins(0,0)
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.savefig("NewDooseA0.0Img" + angles[angleIndex] + ".png", format="png",bbox_inches='tight',pad_inches = 0) 

    print(angleIndex)
    angleIndex = angleIndex+1

1.0
0.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-5.5350678037761e-17..1.659775675675676].


0
1.0
0.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-6.082842576558847e-17..1.4826702702702703].


1
0.972189
0.0


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-7.711206544034293e-17..1.8738648648648648].


4


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000022D019F5E40> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [2]:
#So making a full animation is just going to be PAIN, so instead we'll have it print out the individual frames for said animation instead of trying to hold the entire thing at once. 
#Which would suck.
#Good news, this means we don't have to work with the animation software!

filepathA = r"C:\Users\deran\Desktop\OldLambSim\EsSayeh_w"
wavs = ["0.93", "1.08", "1.27", "1.59", "2.01", "2.69", "2.79", "5.00"]
filepathB = "_p"
angles = ["000", "005", "010", "015", "020", "025", "030", "035", "040", "045", "050", "055", "060", "065", "070", "075",
             "080", "085", "090", "095", "100", "105", "110", "115", "120", "125", "130", "135", "140", "145", "150", "155",
             "160", "165", "170", "175", "180", "185", "190", "195", "200", "205", "210", "215", "220", "225", "230", "235",
             "240", "245", "250", "255", "260", "265", "270", "275", "280", "285", "290", "295", "300", "305", "310", "315", 
             "320", "325", "330", "335", "340", "345", "350", "355"] 
filepathC = r".colorCCD.Jcube.csv"

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
#import mpl_interactions.ipyplot as iplt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage
from pyvims import VIMS
from matplotlib.colors import LogNorm

angleIndex = 0
while (angleIndex < len(angles)):

    filepaths = [
        filepathA + wavs[0] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[1] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[2] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[3] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[4] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[5] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[6] + filepathB + angles[angleIndex] + filepathC,
        filepathA + wavs[7] + filepathB + angles[angleIndex] + filepathC
    ]

    print(filepaths[0])
    
    cubeAxesfps = [
        filepaths[0].removesuffix(".csv") + ".axes.csv",
        filepaths[1].removesuffix(".csv") + ".axes.csv",
        filepaths[2].removesuffix(".csv") + ".axes.csv",
        filepaths[3].removesuffix(".csv") + ".axes.csv",
        filepaths[4].removesuffix(".csv") + ".axes.csv",
        filepaths[5].removesuffix(".csv") + ".axes.csv",
        filepaths[6].removesuffix(".csv") + ".axes.csv",
        filepaths[7].removesuffix(".csv") + ".axes.csv"
    ]

    print(cubeAxesfps[0])

    cubeGeofpIRs = [
        filepaths[0].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[1].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[2].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[3].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[4].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[5].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[6].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv",
        filepaths[7].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.csv"
    ]

    print(cubeGeofpIRs[0])

    cubeGeofpIRaxess = [
        filepaths[0].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[1].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[2].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[3].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[4].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[5].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[6].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv",
        filepaths[7].removesuffix(".colorCCD.Jcube.csv") + "_geo.colorCCD.Jcube.axes.csv"
    ]

    print(cubeGeofpIRaxess[0])
       
    #Skeleton code nabbed from https://realpython.com/python-csv/

    #Step 1: use the axes to determine the size of what we're dealing with.
    
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = [] #z is kind of useless here but we use it anyway so we don't have to adjust the code that much.
    
    with open(cubeAxesfps[0]) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisCube.append(row[i])
                    elif (line_count == 1):
                        yAxisCube.append(row[i])
                    elif (line_count == 2):
                        zAxisCube.append(row[i])
                    i = i+1
                line_count += 1
    
    with open(cubeGeofpIRaxess[0]) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisGeoIR.append(row[i])
                    elif (line_count == 1):
                        yAxisGeoIR.append(row[i])
                    elif (line_count == 2):
                        zAxisGeoIR.append(row[i])
                    i = i+1
                line_count += 1
    
    #wvlns: (alt: 2) contains the wavelengths, equivalent to our zAxis (Wait I think we don't have this anymore...)
    #Shape contains cube size. Probably wxy, but may be wyx? NB is bands (w) and NL is lines (x or y?)
    #lat, lon: position! 
    #ground_inc, ground_eme, ground_azi: incidence and emission AND azimuth
        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(8)] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(8)] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 

    w = 0
    while (w < 8):
        with open(filepaths[w]) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            i, j, k = 0, 0, 0
            for row in csv_reader:
                while (i < len(xAxisCube)):
                    cubeData[i][j][w] = float(row[i])
                    if (math.isnan(cubeData[i][j][w])):
                        cubeData[i][j][w] = 0 #We set nans to zero to allow plotting to take place, careful!
                    elif (cubeData[i][j][w] < 0):
                        cubeData[i][j][w] = 0 #Negative values are nonsense.
                    elif (cubeData[i][j][w] > 1):
                        cubeData[i][j][w] = 1 #Make saturation obvious? Keep it from overloading. 
                    i = i + 1
                i = 0
                j = j + 1
                if (j >= len(yAxisCube)):
                    j = 0
                    k = k + 1
                line_count += 1
        
        with open(cubeGeofpIRs[w]) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            i, j, k = 0, 0, 0
            for row in csv_reader:
                while (i < len(xAxisGeoIR)):
                    geoIRData[i][j][w] = float(row[i])
                    if (math.isnan(geoIRData[i][j][w])):
                        geoIRData[i][j][w] = 0 #We set nans to zero to allow plotting to take place, careful!
                    elif (geoIRData[i][j][w] < -1000):
                        geoIRData[i][j][w] = 0 #The default value is an extremely negative number. Scrub it.
                    i = i + 1
                i = 0
                j = j + 1
                if (j >= len(yAxisGeoIR)):
                    j = 0
                    k = k + 1
                line_count += 1
        w = w+1
    
    #helpful declarations
    #Wavelength of windows in microns (um)
    windowum = [0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576]
    #Now we need to dynamically find the indeces where this is true. 
    windowInd = [0,1,2,3,4,5,6,7]
        
    def indexFromWav(wav):
        index = 0
        j = 0
        while (j < len(zAxisCube)):
            if (float(zAxisCube[j]) < wav and float(zAxisCube[j+1]) > wav):
                if (wav - float(zAxisCube[j]) < float(zAxisCube[j+1]) - wav):
                    index = j
                    #Have to make sure we pick the closest point, not just the "lower" one.
                else:
                    index = j+1
                j = len(zAxisCube)
            j = j + 1    
        return index
        
    #image dimensions are fungible. Calculate them.  
    xDim = 0
    yDim = 0
    if (len(xAxisCube) > len(yAxisCube)):
        yDim = 9 
        xDim = 9/(len(xAxisCube)/len(yAxisCube))
    else:
        xDim = 9
        yDim = 9/(len(yAxisCube)/len(xAxisCube))
    #This may seem backward, but transpositions occur so...
    
        
    print(np.amax(cubeData))
    print(np.amin(cubeData))
    temp = np.transpose(cubeData)
    
    
    fig, axs = plt.subplots(3,3, figsize = (9,9), layout = "tight") 
    
    #Windows are, according to lambertian_wholedisk:
    #0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576
    
    
    
    c1 = axs[0,0].imshow((temp[windowInd[0]]), cmap="copper")
    c2 = axs[0,1].imshow((temp[windowInd[1]]), cmap="copper")
    c3 = axs[0,2].imshow((temp[windowInd[2]]), cmap="copper")
    c4 = axs[1,0].imshow((temp[windowInd[3]]), cmap="copper")
    c5 = axs[1,2].imshow((temp[windowInd[4]]), cmap="copper")
    c6 = axs[2,0].imshow((temp[windowInd[5]]), cmap="copper")
    c7 = axs[2,1].imshow((temp[windowInd[6]]), cmap="copper")
    c8 = axs[2,2].imshow((temp[windowInd[7]]), cmap="copper")
    #fig.colorbar(c4, ax=cax, label='High') 
    #fig.tight_layout(pad=1.0)
    axs[0,0].axis('off')
    axs[0,1].axis('off')
    axs[0,2].axis('off')
    axs[1,0].axis('off')
    axs[1,1].axis('off')
    axs[1,2].axis('off')
    axs[2,0].axis('off')
    axs[2,1].axis('off')
    axs[2,2].axis('off')
    axs[1,1].axis('off')
    
    #COLOR
    
    mod = 1.
    
    # set R, G, and B
    R = temp[7]/(mod*1.12) #I am still very unsure why we don't have the 1/16 here when we have it in both places in the full thing, but whatever.
    G = temp[4]/(mod*0.22)
    B = (temp[2]-0.03)/(mod*0.37)
    
    #B's value can introduce negatives, remove them.
    i,j = 0,0
    for x in B:
        j=0
        for y in x:
            if (y < 0):
                B[i][j] = 0
            j = j+1
        i = i+1
    
    # Now we need to combine them in the correct form: xyc, that is, x, y, then color value. 
    colorData = [R,G,B] # arranged in cyx. I thought RGB needed to be transposed but evidentialy not. Hmm.
    colorData = np.transpose(colorData) #arranged in xyc
    
    image = axs[1,1].imshow(ndimage.rotate(colorData,90), origin="lower") 
    
    axs[0,0].set(title="0.93 $\mu$m")
    axs[0,1].set(title="1.08 $\mu$m")
    axs[0,2].set(title="1.27 $\mu$m")
    axs[1,0].set(title="1.59 $\mu$m")
    axs[1,1].set(title="bestcolor (5.00, 2.01, 1.27)")
    axs[1,2].set(title="2.01 $\mu$m")
    axs[2,0].set(title="2.69 $\mu$m")
    axs[2,1].set(title="2.79 $\mu$m")
    axs[2,2].set(title="5.00 $\mu$m")
    
    plt.savefig("OldLambAni" + angles[angleIndex] + ".png", format="png")
    print(angleIndex)
    angleIndex = angleIndex+1

<>:273: SyntaxWarning: invalid escape sequence '\m'
<>:274: SyntaxWarning: invalid escape sequence '\m'
<>:275: SyntaxWarning: invalid escape sequence '\m'
<>:276: SyntaxWarning: invalid escape sequence '\m'
<>:278: SyntaxWarning: invalid escape sequence '\m'
<>:279: SyntaxWarning: invalid escape sequence '\m'
<>:280: SyntaxWarning: invalid escape sequence '\m'
<>:281: SyntaxWarning: invalid escape sequence '\m'
<>:273: SyntaxWarning: invalid escape sequence '\m'
<>:274: SyntaxWarning: invalid escape sequence '\m'
<>:275: SyntaxWarning: invalid escape sequence '\m'
<>:276: SyntaxWarning: invalid escape sequence '\m'
<>:278: SyntaxWarning: invalid escape sequence '\m'
<>:279: SyntaxWarning: invalid escape sequence '\m'
<>:280: SyntaxWarning: invalid escape sequence '\m'
<>:281: SyntaxWarning: invalid escape sequence '\m'
C:\Users\deran\AppData\Local\Temp\ipykernel_5948\3473526324.py:273: SyntaxWarning: invalid escape sequence '\m'
  axs[0,0].set(title="0.93 $\mu$m")
C:\Users\deran\AppDa

C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p000.colorCCD.Jcube.csv
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p000.colorCCD.Jcube.axes.csv
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p000_geo.colorCCD.Jcube.csv
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p000_geo.colorCCD.Jcube.axes.csv
0.546363
0.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-3.0991281345776844e-17..1.0671270270270272].


0
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p005.colorCCD.Jcube.csv
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p005.colorCCD.Jcube.axes.csv
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p005_geo.colorCCD.Jcube.csv
C:\Users\deran\Desktop\OldLambSim\EsSayeh_w0.93_p005_geo.colorCCD.Jcube.axes.csv



KeyboardInterrupt



Error in callback <function flush_figures at 0x00000212949C5E40> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [ ]:
#Remember to alter the filepaths to get the view you want!

filepathA = r"C:\Users\deran\Desktop\DooseCSV\2012_A0.0_p"
angles = ["000", "005", "010", "015", "020", "025", "030", "035", "040", "045", "050", "055", "060", "065", "070", "075",
             "080", "085", "090", "095", "100", "105", "110", "115", "120", "125", "130", "135", "140", "145", "150", "155",
             "160", "165", "170", "175", "180", "185", "190", "195", "200", "205", "210", "215", "220", "225", "230", "235",
             "240", "245", "250", "255", "260", "265", "270", "275", "280", "285", "290", "295", "300", "305", "310", "315", 
             "320", "325", "330", "335", "340", "345", "350", "355"] 
filepathB = r"_colorCCD.Jcube.csv"

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
#import mpl_interactions.ipyplot as iplt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage
from pyvims import VIMS
from matplotlib.colors import LogNorm

In [ ]:
%matplotlib ipympl

cubeData = [[[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))] for x in range(len(angles))]
geoIRData = [[[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))] for x in range(len(angles))]

angleIndex = 0
while (angleIndex < len(angles)):
    filepath = filepathA + angles[angleIndex] + filepathB
    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.csv"
    cubeGeofpIRaxes = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.axes.csv"

    #Skeleton code nabbed from https://realpython.com/python-csv/

    #Step 1: use the axes to determine the size of what we're dealing with
    #Yes these are the same every time but I'm not here for efficiency atm.
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []

    with open(cubeAxesfp) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisCube.append(row[i])
                    elif (line_count == 1):
                        yAxisCube.append(row[i])
                    elif (line_count == 2):
                        zAxisCube.append(row[i])
                    i = i+1
                line_count += 1
    
    with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisGeoIR.append(row[i])
                    elif (line_count == 1):
                        yAxisGeoIR.append(row[i])
                    elif (line_count == 2):
                        zAxisGeoIR.append(row[i])
                    i = i+1
                line_count += 1

    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisCube)):
                cubeData[angleIndex][i][j][k] = float(row[i])
                if (math.isnan(cubeData[angleIndex][i][j][k])):
                    cubeData[angleIndex][i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (cubeData[angleIndex][i][j][k] < 0):
                    cubeData[angleIndex][i][j][k] = 0 #Negative values are nonsense.
                elif (cubeData[angleIndex][i][j][k] > 1):
                    cubeData[angleIndex][i][j][k] = 1 #Make saturation obvious? Keep it from overloading. 
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisCube)):
                j = 0
                k = k + 1
            line_count += 1
    
    with open(cubeGeofpIR) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisGeoIR)):
                geoIRData[angleIndex][i][j][k] = float(row[i])
                if (math.isnan(geoIRData[angleIndex][i][j][k])):
                    geoIRData[angleIndex][i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (geoIRData[angleIndex][i][j][k] < -1000):
                    geoIRData[angleIndex][i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisGeoIR)):
                j = 0
                k = k + 1
            line_count += 1
    
    angleIndex = angleIndex + 1

#helpful declarations
#Wavelength of windows in microns (um)
windowum = [0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576]
#Now we need to dynamically find the indeces where this is true. 
windowInd = [0,1,2,3,4,5,6,7]
    
def indexFromWav(wav):
    index = 0
    j = 0
    while (j < len(zAxisCube)):
        if (float(zAxisCube[j]) < wav and float(zAxisCube[j+1]) > wav):
            if (wav - float(zAxisCube[j]) < float(zAxisCube[j+1]) - wav):
                index = j
                #Have to make sure we pick the closest point, not just the "lower" one.
            else:
                index = j+1
            j = len(zAxisCube)
        j = j + 1    
    return index
    
#image dimensions are fungible. Calculate them.  
xDim = 0
yDim = 0
if (len(xAxisCube) > len(yAxisCube)):
    yDim = 9 
    xDim = 9/(len(xAxisCube)/len(yAxisCube))
else:
    xDim = 9
    yDim = 9/(len(yAxisCube)/len(xAxisCube))
#This may seem backward, but transpositions occur so...

    
print(np.amax(cubeData))
print(np.amin(cubeData))
temp = np.transpose(cubeData)

In [ ]:
#NOT RUNNABLE JUST A TEMPLATE
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


fig, axs = plt.subplots(3,3) #matplotlib's axis and figure notation is so annoying and obtuse...

#Now we'd like to try to animate it. 
#https://matplotlib.org/stable/users/explain/animations/animations.html
#Eventually had to use ChatGPT to help here. Could not get the ArtistAnimator to work, but it could
#get the FuncAnimation to work--evidentially defining zz inside the funciton/loop was important.
#Even though I thought I already tried that...

#Now we actually need to get the slices
#Set up x and y divisions.        
#xx = np.linspace(0, 100, 21)
#yy = np.linspace(0, 90, 19)

#The scale value for how much color detail we want:
#scale = 0.04
cmap = 'copper'

c1 = axs[0,0].imshow((temp[windowInd[0]]), cmap="copper")
c2 = axs[0,1].imshow((temp[windowInd[1]]), cmap="copper")
c3 = axs[0,2].imshow((temp[windowInd[2]]), cmap="copper")
c4 = axs[1,0].imshow((temp[windowInd[3]]), cmap="copper")
c5 = axs[1,2].imshow((temp[windowInd[4]]), cmap="copper")
c6 = axs[2,0].imshow((temp[windowInd[5]]), cmap="copper")
c7 = axs[2,1].imshow((temp[windowInd[6]]), cmap="copper")
c8 = axs[2,2].imshow((temp[windowInd[7]]), cmap="copper")
axs[0,0].axis('off')
axs[0,1].axis('off')
axs[0,2].axis('off')
axs[1,0].axis('off')
axs[1,1].axis('off')
axs[1,2].axis('off')
axs[2,0].axis('off')
axs[2,1].axis('off')
axs[2,2].axis('off')
axs[1,1].axis('off')
mod = 1.
# set R, G, and B
R = temp[7]/(mod*1.12) #I am still very unsure why we don't have the 1/16 here when we have it in both places in the full thing, but whatever.
G = temp[4]/(mod*0.22)
B = (temp[2]-0.03)/(mod*0.37)
#B's value can introduce negatives, remove them.
i,j = 0,0
for x in B:
    j=0
    for y in x:
        if (y < 0):
            B[i][j] = 0
        j = j+1
    i = i+1
# Now we need to combine them in the correct form: xyc, that is, x, y, then color value. 
colorData = [R,G,B] # arranged in cyx. I thought RGB needed to be transposed but evidentialy not. Hmm.
colorData = np.transpose(colorData) #arranged in xyc
image = axs[1,1].imshow(ndimage.rotate(colorData,90), origin="lower") 
axs[0,0].set(title="0.93 $\mu$m")
axs[0,1].set(title="1.08 $\mu$m")
axs[0,2].set(title="1.27 $\mu$m")
axs[1,0].set(title="1.59 $\mu$m")
axs[1,1].set(title="bestcolor (5.00, 2.01, 1.27)")
axs[1,2].set(title="2.01 $\mu$m")
axs[2,0].set(title="2.69 $\mu$m")
axs[2,1].set(title="2.79 $\mu$m")
axs[2,2].set(title="5.00 $\mu$m")

fig.colorbar(c4, ax=cax, label='Difference') 
fig.tight_layout(pad=1.0)
fig.set_size_inches(10, 10)
axs[1,1].axis('off')
cax.axis('off')

def update(frame):
    axs[0,0].clear()
    axs[0,1].clear()
    axs[0,2].clear()
    axs[1,0].clear()
    axs[1,2].clear()
    axs[2,0].clear()
    axs[2,1].clear()
    axs[2,2].clear()
    zz1 = np.zeros((Y, X))
    zz2 = np.zeros((Y, X))
    zz3 = np.zeros((Y, X))
    zz4 = np.zeros((Y, X))
    zz5 = np.zeros((Y, X))
    zz6 = np.zeros((Y, X))
    zz7 = np.zeros((Y, X))
    zz8 = np.zeros((Y, X))
    for x in range(X):
        for y in range(Y):
            zz1[y][x] = dataStorage1[x][y][frame]
            zz2[y][x] = dataStorage2[x][y][frame]
            zz3[y][x] = dataStorage3[x][y][frame]
            zz4[y][x] = dataStorage4[x][y][frame]
            zz5[y][x] = dataStorage5[x][y][frame]
            zz6[y][x] = dataStorage6[x][y][frame]
            zz7[y][x] = dataStorage7[x][y][frame]
            zz8[y][x] = dataStorage8[x][y][frame]
    azimuth1 = axs[0,0].pcolor(xx, yy, zz1, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth2 = axs[0,1].pcolor(xx, yy, zz2, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth3 = axs[0,2].pcolor(xx, yy, zz3, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth4 = axs[1,0].pcolor(xx, yy, zz4, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth5 = axs[1,2].pcolor(xx, yy, zz5, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth6 = axs[2,0].pcolor(xx, yy, zz6, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth7 = axs[2,1].pcolor(xx, yy, zz7, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth8 = axs[2,2].pcolor(xx, yy, zz8, cmap=cmap, vmin=-scale, vmax=scale)
    axs[0,0].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '0.93 um at '+angles[frame]+ '° azimuth')
    axs[0,1].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '1.08 um at '+angles[frame]+ '° azimuth')
    axs[0,2].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '1.27 um at '+angles[frame]+ '° azimuth')
    axs[1,0].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '1.59 um at '+angles[frame]+ '° azimuth')
    axs[1,2].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '2.01 um at '+angles[frame]+ '° azimuth')
    axs[2,0].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '2.69 um at '+angles[frame]+ '° azimuth')
    axs[2,1].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '2.79 um at '+angles[frame]+ '° azimuth')
    axs[2,2].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '5.00 um at '+angles[frame]+ '° azimuth')
    
ani = animation.FuncAnimation(fig, update, frames=Z, interval=100)

# To save the animation as an mp4
#writer = animation.FFMpegFileWriter(fps = 2)
#ani.save('ObservationGeometriesDifference.mp4', writer = writer)
#writer = animation.PillowWriter(fps=5)
#ani.save('TEST2.gif', writer=writer)

plt.show()

In [ ]:
simTest = r"C:\Users\deran\Desktop\DooseCSV\2012_A0.0_p000_colorCCD.Jcube.csv"

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
#import mpl_interactions.ipyplot as iplt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage
from pyvims import VIMS
from matplotlib.colors import LogNorm

In [ ]:
%matplotlib ipympl

customInput = simTest

filepath = customInput #use the name of the cube you want in the above cell.

#Now we extract the axes file as well...
cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"

#And you know what why not all the others as well.
cubeGeofpIR = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.csv"
cubeGeofpIRaxes = filepath.removesuffix("colorCCD.Jcube.csv") + "geo.Jcube.axes.csv"
#Basically, we don't use axesfp if it's not needed.

#Skeleton code nabbed from https://realpython.com/python-csv/
#We need to be able to handle both local and other cubes. 
#Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
#it in the way the code understands.

#Step 1: use the axes to determine the size of what we're dealing with.

xAxisCube = []
yAxisCube = []
zAxisCube = []

xAxisGeoIR = []
yAxisGeoIR = []
zAxisGeoIR = []

with open(cubeAxesfp) as csv_file: #remember to tab.
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            i = 0
            L = len(row)
            while (i < L-1):
                if (line_count == 0):
                    xAxisCube.append(row[i])
                elif (line_count == 1):
                    yAxisCube.append(row[i])
                elif (line_count == 2):
                    zAxisCube.append(row[i])
                i = i+1
            line_count += 1

with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            i = 0
            L = len(row)
            while (i < L-1):
                if (line_count == 0):
                    xAxisGeoIR.append(row[i])
                elif (line_count == 1):
                    yAxisGeoIR.append(row[i])
                elif (line_count == 2):
                    zAxisGeoIR.append(row[i])
                i = i+1
            line_count += 1

#wvlns: (alt: 2) contains the wavelengths, equivalent to our zAxis.
#Shape contains cube size. Probably wxy, but may be wyx? NB is bands (w) and NL is lines (x or y?)
#lat, lon: position! 
#ground_inc, ground_eme, ground_azi: incidence and emission AND azimuth
    
#We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
#The lengths of these arrays tell us how to extract the data.

cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]

#The above holds the data of the cube itself. 

with open(filepath) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    i, j, k = 0, 0, 0
    for row in csv_reader:
        while (i < len(xAxisCube)):
            cubeData[i][j][k] = float(row[i])
            if (math.isnan(cubeData[i][j][k])):
                cubeData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
            elif (cubeData[i][j][k] < 0):
                cubeData[i][j][k] = 0 #Negative values are nonsense.
            elif (cubeData[i][j][k] > 1):
                cubeData[i][j][k] = 1 #Make saturation obvious? Keep it from overloading. 
            i = i + 1
        i = 0
        j = j + 1
        if (j >= len(yAxisCube)):
            j = 0
            k = k + 1
        line_count += 1

with open(cubeGeofpIR) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    i, j, k = 0, 0, 0
    for row in csv_reader:
        while (i < len(xAxisGeoIR)):
            geoIRData[i][j][k] = float(row[i])
            if (math.isnan(geoIRData[i][j][k])):
                geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
            elif (geoIRData[i][j][k] < -1000):
                geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
            i = i + 1
        i = 0
        j = j + 1
        if (j >= len(yAxisGeoIR)):
            j = 0
            k = k + 1
        line_count += 1

#helpful declarations
#Wavelength of windows in microns (um)
windowum = [0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576]
#Now we need to dynamically find the indeces where this is true. 
windowInd = [0,1,2,3,4,5,6,7]
    
def indexFromWav(wav):
    index = 0
    j = 0
    while (j < len(zAxisCube)):
        if (float(zAxisCube[j]) < wav and float(zAxisCube[j+1]) > wav):
            if (wav - float(zAxisCube[j]) < float(zAxisCube[j+1]) - wav):
                index = j
                #Have to make sure we pick the closest point, not just the "lower" one.
            else:
                index = j+1
            j = len(zAxisCube)
        j = j + 1    
    return index
    
#image dimensions are fungible. Calculate them.  
xDim = 0
yDim = 0
if (len(xAxisCube) > len(yAxisCube)):
    yDim = 9 
    xDim = 9/(len(xAxisCube)/len(yAxisCube))
else:
    xDim = 9
    yDim = 9/(len(yAxisCube)/len(xAxisCube))
#This may seem backward, but transpositions occur so...

    
print(np.amax(cubeData))
print(np.amin(cubeData))
temp = np.transpose(cubeData)

In [ ]:

fig, axs = plt.subplots(3,3, figsize = (9,9), layout = "tight") 

#Windows are, according to lambertian_wholedisk:
#0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576



c1 = axs[0,0].imshow((temp[windowInd[0]]), cmap="copper")
c2 = axs[0,1].imshow((temp[windowInd[1]]), cmap="copper")
c3 = axs[0,2].imshow((temp[windowInd[2]]), cmap="copper")
c4 = axs[1,0].imshow((temp[windowInd[3]]), cmap="copper")
c5 = axs[1,2].imshow((temp[windowInd[4]]), cmap="copper")
c6 = axs[2,0].imshow((temp[windowInd[5]]), cmap="copper")
c7 = axs[2,1].imshow((temp[windowInd[6]]), cmap="copper")
c8 = axs[2,2].imshow((temp[windowInd[7]]), cmap="copper")
#fig.colorbar(c4, ax=cax, label='High') 
#fig.tight_layout(pad=1.0)
axs[0,0].axis('off')
axs[0,1].axis('off')
axs[0,2].axis('off')
axs[1,0].axis('off')
axs[1,1].axis('off')
axs[1,2].axis('off')
axs[2,0].axis('off')
axs[2,1].axis('off')
axs[2,2].axis('off')
axs[1,1].axis('off')

#COLOR

mod = 1.

# set R, G, and B
R = temp[7]/(mod*1.12) #I am still very unsure why we don't have the 1/16 here when we have it in both places in the full thing, but whatever.
G = temp[4]/(mod*0.22)
B = (temp[2]-0.03)/(mod*0.37)

#B's value can introduce negatives, remove them.
i,j = 0,0
for x in B:
    j=0
    for y in x:
        if (y < 0):
            B[i][j] = 0
        j = j+1
    i = i+1

# Now we need to combine them in the correct form: xyc, that is, x, y, then color value. 
colorData = [R,G,B] # arranged in cyx. I thought RGB needed to be transposed but evidentialy not. Hmm.
colorData = np.transpose(colorData) #arranged in xyc

image = axs[1,1].imshow(ndimage.rotate(colorData,90), origin="lower") 

axs[0,0].set(title="0.93 $\mu$m")
axs[0,1].set(title="1.08 $\mu$m")
axs[0,2].set(title="1.27 $\mu$m")
axs[1,0].set(title="1.59 $\mu$m")
axs[1,1].set(title="bestcolor (5.00, 2.01, 1.27)")
axs[1,2].set(title="2.01 $\mu$m")
axs[2,0].set(title="2.69 $\mu$m")
axs[2,1].set(title="2.79 $\mu$m")
axs[2,2].set(title="5.00 $\mu$m")

plt.show()

In [ ]:
fig, axs = plt.subplots(3,3, figsize = (9,9), layout="tight") 

#First, we need to find the maximum of all the data. This will take some finagling.
allData = temp #vestigal assignment from old code.
maximum =  0.333

#COLOR

c1 = axs[0,0].imshow(temp[windowInd[0]], cmap="copper",vmax = maximum)
c2 = axs[0,1].imshow(temp[windowInd[1]], cmap="copper",vmax = maximum)
c3 = axs[0,2].imshow(temp[windowInd[2]], cmap="copper",vmax = maximum)
c4 = axs[1,0].imshow(temp[windowInd[3]], cmap="copper",vmax = maximum)
c5 = axs[1,2].imshow(temp[windowInd[4]], cmap="copper",vmax = maximum)
c6 = axs[2,0].imshow(temp[windowInd[5]], cmap="copper",vmax = maximum)
c7 = axs[2,1].imshow(temp[windowInd[6]], cmap="copper",vmax = maximum)
c8 = axs[2,2].imshow(temp[windowInd[7]], cmap="copper",vmax = maximum)
image = axs[1,1].imshow(ndimage.rotate(colorData,90), origin="lower") 

#fig.colorbar(c4, ax=cax, label='High') 
#fig.tight_layout(pad=1.0)
axs[0,0].axis('off')
axs[0,1].axis('off')
axs[0,2].axis('off')
axs[1,0].axis('off')
axs[1,1].axis('off')
axs[1,2].axis('off')
axs[2,0].axis('off')
axs[2,1].axis('off')
axs[2,2].axis('off')



axs[0,0].set(title="0.93 $\mu$m")
axs[0,1].set(title="1.08 $\mu$m")
axs[0,2].set(title="1.27 $\mu$m")
axs[1,0].set(title="1.59 $\mu$m")
axs[1,1].set(title="bestcolor (5.00, 2.01, 1.27)")
axs[1,2].set(title="2.01 $\mu$m")
axs[2,0].set(title="2.69 $\mu$m")
axs[2,1].set(title="2.79 $\mu$m")
axs[2,2].set(title="5.00 $\mu$m")

plt.show()

In [ ]:
%matplotlib ipympl

fig, ax = plt.subplots(figsize = (xDim,yDim))

#Data storage is a 3D array in the wrong order, we need to get a 2D one out of it.
temp = np.transpose(cubeData) #xyz to zyx
dataShaped = np.transpose(temp[0]) #zyx to yx to xy

image = ax.pcolormesh(dataShaped, cmap="copper")
ax.set(xticks=[], yticks=[], title = 'INIT') #Remove ticks.

def update(frame):
    ax.clear()
    dataShaped = np.transpose(temp[frame]) #zyx to yx to xy
    image = ax.pcolormesh(dataShaped, cmap="copper")
    ax.set(xticks=[], yticks=[], title = 'Wavelength: ' + str(zAxisCube[frame]) + ' $\mu$m High: ' + str(np.amax(dataShaped)))
    
ani = animation.FuncAnimation(fig, update, frames=len(zAxisCube), interval=100)

plt.show()

In [ ]:
# CELL 3: Stop the animation. Always do this before re-running cell 2.

ani.pause()
plt.close()

In [ ]:
#NOT RUNNABLE JUST A TEMPLATE

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


fig, axs = plt.subplots(3,3) #matplotlib's axis and figure notation is so annoying and obtuse...

#Now we'd like to try to animate it. 
#https://matplotlib.org/stable/users/explain/animations/animations.html
#Eventually had to use ChatGPT to help here. Could not get the ArtistAnimator to work, but it could
#get the FuncAnimation to work--evidentially defining zz inside the funciton/loop was important.
#Even though I thought I already tried that...

#Now we actually need to get the slices
#Set up x and y divisions.        
xx = np.linspace(0, 100, 21)
yy = np.linspace(0, 90, 19)

#The scale value for how much color detail we want:
scale = 0.04
cmap = 'seismic'

c1 = axs[0,0].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c2 = axs[0,1].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c3 = axs[0,2].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c4 = axs[1,0].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c5 = axs[1,2].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c6 = axs[2,0].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c7 = axs[2,1].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
c8 = axs[2,2].pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap, vmin=-scale, vmax=scale)
cax = axs[1,1].inset_axes([0.0, 0.1, 0.5, 1.0])
fig.colorbar(c4, ax=cax, label='Difference') 
fig.tight_layout(pad=1.0)
fig.set_size_inches(10, 10)
axs[1,1].axis('off')
cax.axis('off')
#fig.suptitle("Specular Minus Lambertian")

# annoyance of animation: trying to update the color bar duplicates it.
# so we need a "dummy" plot to initialize it before we start animating. 

#Making an array so the title can display the azimuth angle. 
angles = ["000", "005", "010", "015", "020", "025", "030", "035", "040", "045", "050", "055", "060", "065", "070", "075",
             "080", "085", "090", "095", "100", "105", "110", "115", "120", "125", "130", "135", "140", "145", "150", "155",
             "160", "165", "170", "175", "180"] 

def update(frame):
    axs[0,0].clear()
    axs[0,1].clear()
    axs[0,2].clear()
    axs[1,0].clear()
    axs[1,2].clear()
    axs[2,0].clear()
    axs[2,1].clear()
    axs[2,2].clear()
    zz1 = np.zeros((Y, X))
    zz2 = np.zeros((Y, X))
    zz3 = np.zeros((Y, X))
    zz4 = np.zeros((Y, X))
    zz5 = np.zeros((Y, X))
    zz6 = np.zeros((Y, X))
    zz7 = np.zeros((Y, X))
    zz8 = np.zeros((Y, X))
    for x in range(X):
        for y in range(Y):
            zz1[y][x] = dataStorage1[x][y][frame]
            zz2[y][x] = dataStorage2[x][y][frame]
            zz3[y][x] = dataStorage3[x][y][frame]
            zz4[y][x] = dataStorage4[x][y][frame]
            zz5[y][x] = dataStorage5[x][y][frame]
            zz6[y][x] = dataStorage6[x][y][frame]
            zz7[y][x] = dataStorage7[x][y][frame]
            zz8[y][x] = dataStorage8[x][y][frame]
    azimuth1 = axs[0,0].pcolor(xx, yy, zz1, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth2 = axs[0,1].pcolor(xx, yy, zz2, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth3 = axs[0,2].pcolor(xx, yy, zz3, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth4 = axs[1,0].pcolor(xx, yy, zz4, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth5 = axs[1,2].pcolor(xx, yy, zz5, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth6 = axs[2,0].pcolor(xx, yy, zz6, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth7 = axs[2,1].pcolor(xx, yy, zz7, cmap=cmap, vmin=-scale, vmax=scale)
    azimuth8 = axs[2,2].pcolor(xx, yy, zz8, cmap=cmap, vmin=-scale, vmax=scale)
    axs[0,0].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '0.93 um at '+angles[frame]+ '° azimuth')
    axs[0,1].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '1.08 um at '+angles[frame]+ '° azimuth')
    axs[0,2].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '1.27 um at '+angles[frame]+ '° azimuth')
    axs[1,0].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '1.59 um at '+angles[frame]+ '° azimuth')
    axs[1,2].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '2.01 um at '+angles[frame]+ '° azimuth')
    axs[2,0].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '2.69 um at '+angles[frame]+ '° azimuth')
    axs[2,1].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '2.79 um at '+angles[frame]+ '° azimuth')
    axs[2,2].set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = '5.00 um at '+angles[frame]+ '° azimuth')
    
ani = animation.FuncAnimation(fig, update, frames=Z, interval=100)

# To save the animation as an mp4
#writer = animation.FFMpegFileWriter(fps = 2)
#ani.save('ObservationGeometriesDifference.mp4', writer = writer)
#writer = animation.PillowWriter(fps=5)
#ani.save('TEST2.gif', writer=writer)

plt.show()